In [202]:
# the target is to gernerate a series of measurement script to qasm and
# then to analyse them.
# First function should be submission circuit
#from api_wrappers import HoneywellQAPI as QAPI
#import time
#import config
from scipy.optimize import minimize
import numpy as np
import qiskit as qk
import networkx as nx
import tenpy
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
# custom things
from isonetwork import IsoTensor, IsoNetwork, QKParamCircuit
import mps
import networks
import random

##useless part for Daoheng:
def fh_bases_gen(s,t,l):
    bases = ''
    for i in range(l):
        if i>s-1 and i < t:
            bases += 'b'
        else: 
            bases += 'a'
    return bases

def mps_meausrement(bases, FH, shots, preg, breg, pcircs):
    start = time.time()
    # run job and retrieve data
    psi_mps = IsoMPS(preg,breg,pcircs,bases=bases,FH=FH)
    openqasm = psi_mps
    job_id = qapi.submit_job(openqasm, shots=shots, name='repetition code') 
    jr = qapi.retrieve_job(job_id)


'''
# run job and retrieve data
job_id = qapi.submit_job(openqasm, shots=shots, name='repetition code') 
jr = qapi.retrieve_job(job_id)
print('Total time = {0:1.4f} s'.format(time.time() - start))
print('\nRaw output:')
print(jr)'''

"\n# run job and retrieve data\njob_id = qapi.submit_job(openqasm, shots=shots, name='repetition code') \njr = qapi.retrieve_job(job_id)\nprint('Total time = {0:1.4f} s'.format(time.time() - start))\nprint('\nRaw output:')\nprint(jr)"

In [203]:
'''
generate measurement bases for a spinless fermionic chain pauli basis correlator
(adding the Jordan wigner string)
the measurement bases information output in a list: including a Pauli string which specifies 
the measurement bases for each site, and a number, which is a constant factor, specifies the 
weight of that correlator
'''
# follow the convention of writing a correlator, we are measuring corr_pauli[0]_{s} corr_pauli[1]_{t}
def fh_bases_gen(s,t,l,corr_pauli):
    factor = 1
    bases = ''
    if s != t:
        for i in range(l):
            if i == t - 1:
                bases += corr_pauli[0]
            elif i == s - 1:
                if corr_pauli[1] == 'x':
                    pauli_o = 'y'
                elif corr_pauli[1] == 'y':
                    pauli_o = 'x'
                bases += pauli_o
                factor = 1j
            else:
                bases += 'z'
    else:
        for i in range(l):
        # s == t, corr_pauli = 'x', 'y', 'z' form
            if i == t - 1:
                bases += corr_pauli[0]
            else:
                bases += 'z'
    res = [bases, factor]
    return res

'''
generate measurement bases for a green function of a spinless fermionic chain
output as a list of lists, with each list contains a pauli string, and a constant 
factor, each list correspond to a certain (type of) measurement
'''
def fermion_corr_to_pauli(total_length, corr_list):
    # corr_list specifies a Green function: for example, [3,'^',1,'_'] means c^{dagger}_3 c_1
    base_list = []
    if corr_list[1] == '^':
        list1 = [1/2, 1/2 * 1j]
    else:
        list1 = [1/2, -1/2 * 1j]
    if corr_list[3] == '^':
        list2 = [1/2, 1/2 * 1j]
    else:
        list2 = [1/2, -1/2 * 1j]
    factor_list = [list1[0]* list2[0], list1[0]* list2[1], list1[1]* list2[0], list1[1]* list2[1]]
    corr_list1 = ['xx', 'xy', 'yx', 'yy']
    for k in range(4):
        ori_base = fh_bases_gen(int(corr_list[0]), int(corr_list[2]), total_length, corr_list1[k])
        base_list.append([ori_base[0], ori_base[1] * factor_list[k]])
    if corr_list[0] == corr_list[2]:
        base_list = []
        base_list.append([fh_bases_gen(corr_list[0],corr_list[2],total_length,'x')[0], 0.25])
        base_list.append([fh_bases_gen(corr_list[0],corr_list[2],total_length,'y')[0], 0.25])
        base_list.append([fh_bases_gen(corr_list[0],corr_list[2],total_length,'z')[0], 0.5])        
    return base_list

'''
generate measurement bases for a green function of a spinful fermionic chain
output as a list of lists, with each list contains a pauli string, and a constant 
factor, each list correspond to a certain (type of) measurement
'''
def spinful_fermion_corr(corr_string, total_len):
    # map n spin-1/2 fermion to 2n qubits, follow the convention, 1up, 1down, 2up, 2down, 3up, 3down
    # refer to Open Fermion library.
    # corr_string sample 2^u4_d, 2^n4_n, 2^n2_n, 2Nn4Nn, 2^x4^x
    site_map = [int(corr_string[0]), int(corr_string[3])]
    spin_sector_list = [corr_string[2], corr_string[5]]
    site_map_new =[]
    JW_site = []
    base_list = []
    for k in range(2):
        if spin_sector_list[k] == 'u':
            site_map_new.append((site_map[k] - 1) * 2)
        elif spin_sector_list[k] == 'd':
            site_map_new.append((site_map[k] - 1) * 2 + 1)
        elif spin_sector_list[k] == 'n':
            site_map_new.append((site_map[k] - 1) * 2)
            site_map_new.append((site_map[k] - 1) * 2 + 1)
    if len(site_map_new) == 2:
        if corr_string[1] == '^':
            list1 = [1/2, 1/2 * 1j]
        else:
            list1 = [1/2, -1/2 * 1j]
        if corr_string[3] == '^':
            list2 = [1/2, 1/2 * 1j]
        else:
            list2 = [1/2, -1/2 * 1j]
        factor_list = [list1[0]* list2[0], list1[0]* list2[1], list1[1]* list2[0], list1[1]* list2[1]]
        corr_list = ['xx', 'xy', 'yx', 'yy']
        for m in range(len(corr_list)):
            #base = 2 * total_len * 'z'
            corr_pauli = corr_list[m]                    
            if corr_pauli[0] == 'x':
                pauli_o = 'y'
            elif corr_pauli[0] == 'y':
                pauli_o = 'x'
            pauli_o1 = corr_pauli[1]
            if site_map_new[0] > site_map_new[1]:
                pauli_o_list = [pauli_o1, pauli_o]
            else:
                pauli_o_list = [pauli_o, pauli_o1]                
            factor = 1j
            base = site_map_new[0] * 'z' + corr_pauli[0] + (site_map_new[1] - site_map_new[0]) * 'z' +\
            pauli_o + (2*total_len - site_map_new[1]) * 'z'
            factor = factor * factor_list[m]
            base_list.append([base, factor])
    return base_list

In [204]:
print(fh_bases_gen(2,2,10,'xx'))
fh_bases = fh_bases_gen(1,4,10,'xx')
bases1 = fh_bases[0]
print(bases1)
print(fermion_corr_to_pauli(10, [4,'^',2,'_']))
print(spinful_fermion_corr('2^u4_d', 10))

['zxzzzzzzzz', 1]
yzzxzzzzzz
[['zxzyzzzzzz', 0.25j], ['zxzxzzzzzz', (0.25+0j)], ['zyzyzzzzzz', (-0.25+0j)], ['zyzxzzzzzz', 0.25j]]
[['zzxzzzzzyzzzzzzzzzzzzz', 0.25j], ['zzxzzzzzyzzzzzzzzzzzzz', (0.25+0j)], ['zzyzzzzzxzzzzzzzzzzzzz', (-0.25+0j)], ['zzyzzzzzxzzzzzzzzzzzzz', 0.25j]]


In [310]:
# vals is the list of (varational) parameters (which already been calculated)
# also vals is already rearranged to correspond to plabels follow the circuit structure
# in the sequence starting from the last site
def free_fermion_correlator(site_num, block_size, vals, shots, correlator_list):
    # set up bond qubit number and length of unit cell, for simplification, now just set 
    # L = 1, and l_uc equal the number of sites
    l_uc = site_num
    # set up qiskit simulator
    simulator = qk.providers.aer.QasmSimulator(method='matrix_product_state')
    correlator_list_fix = correlator_list
    correlator_list_fix[0] = l_uc + 1 - correlator_list[0]
    correlator_list_fix[2] = l_uc + 1 - correlator_list[2]
    
    # measurement bases part: get the generated bases then reverse its order
    bases_list = fermion_corr_to_pauli(l_uc, correlator_list_fix)
    start = int(correlator_list[2]) - 1
    end = int(correlator_list[0]) - 1
    if start > end:
        temp = start
        start = end
        end = temp
    total_result = 0
    # the outer loop, determine the bases for measurement each time
    for base_j in range(len(bases_list)):
        circs =qk.QuantumCircuit(block_size, l_uc)
        correlation_result = 0
        # extract the j-th bases from the bases list, which is a list of list consists of 
        # Pauli strings and the weight (a constant)
        bases_ele = bases_list[base_j]
        bases_ori = bases_ele[0]
        bases0 = bases_ori
        #bases0 = ''
        # reserve the order of original bases (which follows site sequence) here
        #for base_m in range(len(bases_ori)):
        #    bases0 += bases_ori[len(bases_ori) - base_m - 1]
        plabel_ind = 0
        for j in range(l_uc):
            if j == 0:
                for x_b in range(0, block_size):
                    if (l_uc - 1 - x_b) % 2 == 0:
                        circs.x(x_b)
                for k in range(1, block_size):
                    for m in range(k, 0, -1):
                        circs.rz(-np.pi / 4, m)
                        circs.rz(np.pi / 4, m-1)
                        circs.rx(np.pi / 2, m)
                        circs.ry(np.pi / 2, m-1)
                        circs.cz(m, m-1)
                        circs.rx(-1 * vals[plabel_ind], m)
                        circs.ry(1 * vals[plabel_ind], m-1)
                        circs.cz(m, m-1)
                        circs.rx(-np.pi / 2, m)
                        circs.ry(-np.pi / 2, m-1)
                        circs.rz(np.pi / 4, m)
                        circs.rz(-np.pi / 4, m-1)
                        plabel_ind += 1

            elif j in range(1, l_uc - block_size + 1):
                if (l_uc - 1 - (j + block_size - 1)) % 2 == 0:
                    # NOT the one which just reset
                    circs.x((j-1) % block_size)
                for m in range(block_size - 1, 0, -1):
                    circs.rz(-np.pi / 4, (j + m) % block_size)
                    circs.rz(np.pi / 4, (j + m - 1) % block_size)
                    circs.rx(np.pi / 2, (j + m) % block_size)
                    circs.ry(np.pi / 2, (j + m - 1) % block_size)
                    circs.cz((j + m) % block_size, (j + m - 1) % block_size)
                    circs.rx(-1 * vals[plabel_ind], (j + m) % block_size)
                    circs.ry(1 * vals[plabel_ind], (j + m - 1) % block_size)
                    circs.cz( (j + m) % block_size, (j + m -1) % block_size)
                    circs.rx(-np.pi / 2, (j + m) % block_size)
                    circs.ry(-np.pi / 2, (j + m - 1) % block_size)
                    circs.rz(np.pi / 4, (j + m) % block_size)
                    circs.rz(-np.pi / 4, (j + m - 1) % block_size)
                    plabel_ind += 1
            
            # measure (consider measurement basis)
            if bases0[j] == 'z':
                circs.measure([j % block_size], [j])
            elif bases0[j] == 'x':
                circs.h(j % block_size)
                circs.measure([j % block_size], [j])
            elif bases0[j] == 'y':
                circs.h(j % block_size)
                circs.sdg(j % block_size)
                circs.measure([j % block_size], [j])
            # reset
            circs.reset(j % block_size)
            # update plabel_ind
        #print(circs.draw())
        #circs.draw('mpl', scale = 0.3)
        # after finishing setting up circ, do the measurement
        result = execute(circs, simulator, shots = shots, memory = True).result()
        count1 = result.get_counts(circs)
        #print(count1)
        
        # post analysis:
        # first get the total number of y operator in the basis (need how many 1j when mapping qubit
        # measurement result to the eigenvalue)
        y_num = 0
        for ms in [bases_ori[start], bases_ori[end]]:
            if ms == 'y':
                y_num += 1
        
        # analyze the count result
        for k,v in count1.items():
            res_1time = v * (1j) ** y_num * (1.0 - 2.0 * np.float(k.replace(" ", "")[start] == '1')) * \
            (1.0 - 2.0 * np.float(k.replace(" ", "")[end] == '1')) / shots
            
            # add the JW-string:
            for string_ind in range(start + 1, end):
                res_1time *= (1.0 - 2.0 * np.float(k.replace(" ", "")[string_ind] == '1'))
            # add the statistic of one particular result to the total distribution
            correlation_result += res_1time
        
        # add to the total correlation (a Green function) by each part (XX, YY, XY, YX) and their weight
        total_result += correlation_result * bases_ele[1]
    return total_result

In [306]:
val = [-1.8806916495947272, 3.1415926535897927, -1.880691649594729, \
       3.141592653589793, -2.356194490192345, 2.0344439357957023, -1.570796326794895]

In [312]:
site_num = 5
block_size = 3
shots = 2000
correlator_list = [2,'^',4,'_']
free_fermion_correlator(site_num, block_size, val, shots, correlator_list)

(0.0014999999999999875+0j)

In [313]:
site_num = 20
block_size = 4
shots = 2000
#correlator_list = [6,'^',7,'_']
#print(free_fermion_correlator(site_num, block_size, val, shots, correlator_list))
for i in range(6,10):
    for j in range(6,10):
        correlator_list = [i,'^',j,'_']
        print(free_fermion_correlator(site_num, block_size, val, shots, correlator_list))

(0.4999999999999771+0j)
(-0.008750000000000008+0j)
(0.007250000000000005+0j)
(0.0085+0j)
(-0.0017500000000000154+0j)
(0.4999999999999769+0j)
(-0.0020000000000000018+0j)
(0.007250000000000002+0j)
(-0.01150000000000001+0j)
(-0.0002500000000000002+0j)
(0.4999999999999767+0j)
(0.009500000000000008+0j)
(-0.016750000000000008+0j)
(-0.007250000000000005+0j)
(-0.0030000000000000027+0j)
(0.4999999999999762+0j)


In [20]:
#sample circuit
cir =qk.QuantumCircuit(2,2)
#define initial circ

l_uc = 20
block_size = 4

#for layer in range(1,l_uc-block_size):
#    cir.h((0-layer)%block_size)
#    cir.cz((0-layer)%block_size, (1-layer)%block_size)
#    cir.h(1)
    
cir.h(0)
cir.cz(0,1)
cir.h(1)
cir.measure([0,1],[0,1])
cir.reset(1)
cir.draw()
#simulator = QasmSimulator(method='matrix_product_state')
#result = execute(cir, simulator, shots=2000, memory=True).result()


┌───┐        ┌─┐        
q_0: ┤ H ├─■──────┤M├────────
     └───┘ │ ┌───┐└╥┘┌─┐     
q_1: ──────■─┤ H ├─╫─┤M├─|0>─
             └───┘ ║ └╥┘     
c: 2/══════════════╩══╩══════
                   0  1

In [445]:
for a,b in result.get_counts().items(): print(a,b)

00 518
01 475
10 493
11 514
